In [32]:
import pandas as pd
import numpy as np
import os
from sys import stdout

In [2]:
data = pd.read_csv('../csv_data/cleaned_ord_5.csv')
data.head()

,pos,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,...,N2,P2,Q2,R2,S2,T2,V2,W2,Y2,-2
0,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [37]:
data.columns.values

array(['pos', 'A-2', 'C-2', 'D-2', 'E-2', 'F-2', 'G-2', 'H-2', 'I-2',
       'K-2', 'L-2', 'M-2', 'N-2', 'P-2', 'Q-2', 'R-2', 'S-2', 'T-2',
       'V-2', 'W-2', 'Y-2', '--2', 'A-1', 'C-1', 'D-1', 'E-1', 'F-1',
       'G-1', 'H-1', 'I-1', 'K-1', 'L-1', 'M-1', 'N-1', 'P-1', 'Q-1',
       'R-1', 'S-1', 'T-1', 'V-1', 'W-1', 'Y-1', '--1', 'A0', 'C0', 'D0',
       'E0', 'F0', 'G0', 'H0', 'I0', 'K0', 'L0', 'M0', 'N0', 'P0', 'Q0',
       'R0', 'S0', 'T0', 'V0', 'W0', 'Y0', '-0', 'A1', 'C1', 'D1', 'E1',
       'F1', 'G1', 'H1', 'I1', 'K1', 'L1', 'M1', 'N1', 'P1', 'Q1', 'R1',
       'S1', 'T1', 'V1', 'W1', 'Y1', '-1', 'A2', 'C2', 'D2', 'E2', 'F2',
       'G2', 'H2', 'I2', 'K2', 'L2', 'M2', 'N2', 'P2', 'Q2', 'R2', 'S2',
       'T2', 'V2', 'W2', 'Y2', '-2'], dtype=object)

In [34]:
all_aminos = []

for i in xrange(data.shape[0]):
    stdout.write("\r%d" % i)
#     stdout.flush()
    all_aminos = all_aminos + get_aminos(data, i)
stdout.write("\n")

134813


In [35]:
from collections import Counter

Counter(all_aminos)

# aas = np.asarray(all_aminos)
# aas.flatten()



Counter({'--1': 30264,
         '--2': 30606,
         '-0': 30085,
         '-1': 30271,
         '-2': 30463,
         'A-1': 8808,
         'A-2': 8771,
         'A0': 8834,
         'A1': 8819,
         'A2': 8805,
         'C-1': 1152,
         'C-2': 1152,
         'C0': 1154,
         'C1': 1153,
         'C2': 1151,
         'D-1': 6257,
         'D-2': 6248,
         'D0': 6271,
         'D1': 6255,
         'D2': 6249,
         'E-1': 7023,
         'E-2': 7001,
         'E0': 7046,
         'E1': 7031,
         'E2': 7017,
         'F-1': 4418,
         'F-2': 4409,
         'F0': 4428,
         'F1': 4423,
         'F2': 4416,
         'G-1': 7538,
         'G-2': 7519,
         'G0': 7533,
         'G1': 7531,
         'G2': 7529,
         'H-1': 2581,
         'H-2': 2565,
         'H0': 2586,
         'H1': 2580,
         'H2': 2570,
         'I-1': 6052,
         'I-2': 6041,
         'I0': 6072,
         'I1': 6057,
         'I2': 6039,
         'K-1': 5802,
         '

In [3]:
def get_aminos(df, i):
    """
    Given the dataframe and index, find the aminos that are "on" (ie 1)
    """
    ser = df.ix[i]
    
    aminos = []
    
    for i, v in enumerate(ser.values):
        if v==1:
            amino = ser.index.values[i]
            aminos.append(amino)
            
    return aminos

In [4]:
l = get_aminos(data, 0)
l

['--2', 'V-1', 'L0', 'S1', 'P2']

In [5]:
def make_prev_row(l):
    """
    Given positional aminos, ex:
    ['--2', 'V-1', 'L0', 'S1', 'P2']
    Reconstruct the missing previous seq
    
    Note: only works on position 2 of a protein
    """
    
    n = len(l)
    
    # get the letters without position
    aas = [v[0] for v in l]
    
    # now shift one
    aas = ['-'] + aas[:n-1]
    
    # generate the positions
    pos = map(str, range(-(n//2), (n//2)+1))
    
    # combine them
    r = [v+p for v, p in zip(aas, pos)]
    
    return r

In [6]:
r0 = make_prev_row(l)
r0

['--2', '--1', 'V0', 'L1', 'S2']

In [7]:
def construct_row(l, df):
    """
    Given positional aminos, ex:
    ['--2', '--1', 'V0', 'L1', 'S2']
    Reconstruct the full dataframe row
    """
    
    cols = df.columns.values
    
    n = df.columns.shape[0]
    
    row = np.zeros(n)
    
    for aa in l:
        i = np.where(aa==cols) # finds the index that matches
        row[i] = 1
        
    # set the position to 1
    row[0] = 1
    
    # convert to simple ints
    row = np.asarray([int(v) for v in row])
        
    return row.reshape(1, -1)

In [8]:
full_r0 = construct_row(r0, data)
full_r0

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [9]:
pd.DataFrame(full_r0, columns=data.columns.values)

,pos,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,...,N2,P2,Q2,R2,S2,T2,V2,W2,Y2,-2
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
# specific to the 5-spacing
def get_middle(df, i):
    return get_aminos(df, i)[2][0]

In [11]:
get_middle(data, 0)

'L'

### Now that we can do it for a single example, let's get the indices

In [12]:
def find_proteins(data):

    i = 1

    indices = []

    start_i = 0

    while i < data.shape[0]:
        curr_pos = data.ix[i]['pos']
        prev_pos = data.ix[i-1]['pos']

        # ie we've transitioned to a new start of a protein
        if prev_pos > curr_pos:
            ind = [start_i, i]
            indices.append(ind)
            start_i = i

        i += 1

    # dont forget to snag the last sequence!
    final = [start_i, data.shape[0]]
    indices.append(final)
    
    return indices

In [13]:
indices = find_proteins(data)

In [15]:
print len(indices)
np.asarray(indices)

347


array([[     0,    140],
       [   140,    550],
       [   550,    695],
       [   695,    856],
       [   856,   1155],
       [  1155,   2968],
       [  2968,   3314],
       [  3314,   3721],
       [  3721,   3883],
       [  3883,   4035],
       [  4035,   4310],
       [  4310,   4372],
       [  4372,   4535],
       [  4535,   4817],
       [  4817,   5270],
       [  5270,   5434],
       [  5434,   5846],
       [  5846,   6040],
       [  6040,   6332],
       [  6332,   6727],
       [  6727,   7216],
       [  7216,   7531],
       [  7531,   7872],
       [  7872,   8110],
       [  8110,   8593],
       [  8593,   9188],
       [  9188,   9252],
       [  9252,   9943],
       [  9943,  10041],
       [ 10041,  10534],
       [ 10534,  10694],
       [ 10694,  10949],
       [ 10949,  11195],
       [ 11195,  11490],
       [ 11490,  11902],
       [ 11902,  12131],
       [ 12131,  12498],
       [ 12498,  12727],
       [ 12727,  13123],
       [ 13123,  13836],


# Example of getting the first protein constructed

In [16]:
protein107 = data.values[indices[107][0]:indices[107][1]]
protein107

array([[   2,    0,    0, ...,    0,    0,    1],
       [   3,    0,    0, ...,    0,    0,    1],
       [   4,    0,    0, ...,    0,    0,    1],
       ..., 
       [3783,    0,    0, ...,    0,    0,    0],
       [3784,    0,    0, ...,    0,    0,    1],
       [3785,    0,    0, ...,    0,    0,    1]])

In [20]:
# fully captured first protein
protein_0 = data.values[indices[0][0]:indices[0][1]]
protein_0

array([[  2,   0,   0, ...,   0,   0,   0],
       [  3,   0,   0, ...,   0,   0,   0],
       [  4,   0,   0, ...,   0,   0,   0],
       ..., 
       [139,   0,   0, ...,   0,   0,   0],
       [140,   0,   0, ...,   0,   0,   1],
       [141,   0,   0, ...,   0,   0,   1]])

In [29]:
# will just change the first index of indices for each protein
protein0_row0 = construct_row(make_prev_row(get_aminos(data, indices[0][0])), data)
protein0_row0

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [39]:
full_protein0 = np.concatenate((protein0_row0, protein_0))
print full_protein0.shape
full_protein0

(141, 106)


array([[  1,   0,   0, ...,   0,   0,   0],
       [  2,   0,   0, ...,   0,   0,   0],
       [  3,   0,   0, ...,   0,   0,   0],
       ..., 
       [139,   0,   0, ...,   0,   0,   0],
       [140,   0,   0, ...,   0,   0,   1],
       [141,   0,   0, ...,   0,   0,   1]])

In [51]:
protein0_df = pd.DataFrame(full_protein0[:, 1:], columns=data.columns[1:], index=full_protein0[:, 0])
protein0_df.head()

,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,L-2,...,N2,P2,Q2,R2,S2,T2,V2,W2,Y2,-2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [194]:
protein0_seq = [get_middle(protein0_df, i) for i in xrange(1, protein0_df.shape[0]+1)]

In [195]:
protein0_seq[:10]

['V', 'L', 'S', 'P', 'A', 'D', 'K', 'T', 'N', 'V']

In [196]:
protein0_seq_df = pd.DataFrame(protein0_seq, columns=['Amino Acid'], index=protein0_df.index)

In [197]:
protein0_seq_df.head()

,Amino Acid
1,V
2,L
3,S
4,P
5,A


In [200]:
protein0_seq_df.to_csv('../proteins/protein0_data.csv')

In [201]:
saved_protein0 = pd.read_csv('../proteins/protein0_data.csv', index_col=0)
saved_protein0

,Amino Acid
1,V
2,L
3,S
4,P
5,A
6,D
7,K
8,T
9,N
10,V


In [202]:
saved_protein0.shape

(141, 1)

In [241]:
one_hot = pd.read_csv('../one_hot_labels.csv')
one_hot.head()

,H,E,T,S,B,U
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [65]:
protein0_labels = one_hot.values[indices[0][0]:indices[0][1]]
protein0_labels

array([[ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0

In [63]:
one_hot.shape

(134815, 6)

In [66]:
protein0_labels.shape

(140, 6)

In [72]:
lr0 = one_hot.values[0].reshape(1, -1)
lr0

array([[ 0.,  0.,  0.,  0.,  0.,  1.]])

In [74]:
protein0_labels = np.concatenate((lr0, protein0_labels))

In [75]:
print protein0_labels.shape
print protein0_df.shape

(141, 6)
(141, 105)


In [80]:
protein0_labels_df = pd.DataFrame(protein0_labels, columns=one_hot.columns, index=xrange(1, 142))
protein0_labels_df.head()

,H,E,T,S,B,U
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0


In [203]:
protein0_labels_df.to_csv('../proteins/protein0_labels.csv')

In [204]:
pd.read_csv('../proteins/protein0_labels.csv', index_col=0)

,H,E,T,S,B,U
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,1,0,0,0,0,0
5,1,0,0,0,0,0
6,1,0,0,0,0,0
7,1,0,0,0,0,0
8,1,0,0,0,0,0
9,1,0,0,0,0,0
10,1,0,0,0,0,0


# Now iterate over all of the indices and construct proteins and full DF

### DONT FORGET --- We need to add these rows to the labels data frame too

In [112]:
proteins = [np.asarray(data.values[s:e]) for s, e in indices]

In [124]:
len(proteins)

347

In [139]:
proteins[0][:10]

array([[ 2,  0,  0, ...,  0,  0,  0],
       [ 3,  0,  0, ...,  0,  0,  0],
       [ 4,  0,  0, ...,  0,  0,  0],
       ..., 
       [ 9,  0,  0, ...,  0,  0,  0],
       [10,  0,  0, ...,  0,  0,  0],
       [11,  0,  0, ...,  0,  0,  0]])

In [130]:
proteins[0].shape

(140, 106)

In [126]:
protein_labels = [np.asarray(one_hot.values[s:e]) for s, e in indices]

In [138]:
len(protein_labels)

347

In [135]:
protein_labels[0][:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.]])

In [129]:
protein_labels[0].shape

(140, 6)

In [131]:
proteins[0][:10]

array([[ 2,  0,  0, ...,  0,  0,  0],
       [ 3,  0,  0, ...,  0,  0,  0],
       [ 4,  0,  0, ...,  0,  0,  0],
       ..., 
       [ 9,  0,  0, ...,  0,  0,  0],
       [10,  0,  0, ...,  0,  0,  0],
       [11,  0,  0, ...,  0,  0,  0]])

In [140]:
construct_row(make_prev_row(get_aminos(data, indices[0][0])), data)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [183]:
get_aminos(data, 0)[2][0]

'L'

In [188]:
get_aminos(label_df, 1)[0]

'U'

In [211]:
protein_label0 = np.asarray([[0, 0, 0, 0, 0, 1]])

for i, (p, pl) in enumerate(zip(proteins, protein_labels)):
    
    # first construct the full protein
    protein_row0 = construct_row(make_prev_row(get_aminos(data, indices[i][0])), data)
    full_protein = np.concatenate((protein_row0, p))
    
    # then construct the protein label
    full_label = np.concatenate((protein_label0, pl))
    
    # construct dataframes with indexes
    protein_df = pd.DataFrame(full_protein[:, 1:], 
                              columns=data.columns[1:], 
                              index=full_protein[:, 0])
    
    label_df = pd.DataFrame(full_label, 
                        columns=one_hot.columns, 
                        index=xrange(1, 1 + full_label.shape[0]))
    
    # make them ints, instead of floats
    label_df[label_df.columns] = label_df[label_df.columns].astype(int)
    
    # make the sequence
    full_seq = [get_middle(protein_df, n) for n in xrange(1, protein_df.shape[0]+1)]
    
    # get into DF
    seq_df = pd.DataFrame(full_seq, columns=['Amino Acid'], index=protein_df.index)
    
    # construct filenames
    data_filename = '../proteins/protein' + str(i) + '_data.csv'
    label_filename = '../proteins/protein' + str(i) + '_labels.csv' 
    
    # finally, save
    seq_df.to_csv(data_filename)
    label_df.to_csv(label_filename)


In [212]:
last_protein = pd.read_csv('../proteins/protein346_data.csv', index_col=0)
last_protein.head()

,Amino Acid
1,Y
2,D
3,F
4,K
5,K


In [213]:
last_protein_labels = pd.read_csv('../proteins/protein346_labels.csv', index_col=0)
last_protein_labels.head()

,H,E,T,S,B,U
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0


In [214]:
last_protein.shape

(499, 1)

In [215]:
last_protein_labels.shape

(499, 6)

In [225]:
# check that we still have the right breakdown
d = np.diff(indices[-1])+1
d[0]

499

In [218]:
last_protein.values

array([['Y'],
       ['D'],
       ['F'],
       ['K'],
       ['K'],
       ['I'],
       ['N'],
       ['N'],
       ['L'],
       ['R'],
       ['G'],
       ['I'],
       ['E'],
       ['R'],
       ['E'],
       ['T'],
       ['L'],
       ['R'],
       ['V'],
       ['T'],
       ['D'],
       ['C'],
       ['G'],
       ['N'],
       ['L'],
       ['A'],
       ['T'],
       ['S'],
       ['N'],
       ['H'],
       ['P'],
       ['D'],
       ['G'],
       ['L'],
       ['G'],
       ['H'],
       ['K'],
       ['L'],
       ['T'],
       ['N'],
       ['N'],
       ['S'],
       ['I'],
       ['T'],
       ['V'],
       ['D'],
       ['F'],
       ['S'],
       ['E'],
       ['N'],
       ['L'],
       ['L'],
       ['E'],
       ['L'],
       ['I'],
       ['T'],
       ['K'],
       ['P'],
       ['H'],
       ['D'],
       ['S'],
       ['I'],
       ['D'],
       ['K'],
       ['A'],
       ['I'],
       ['G'],
       ['E'],
       ['L'],
       ['Y'],
       ['Q'],
      

In [219]:
data.values[indices[-1][0]:indices[-1][1]]

array([[  2,   0,   0, ...,   0,   0,   0],
       [  3,   0,   0, ...,   0,   0,   0],
       [  4,   0,   0, ...,   0,   0,   0],
       ..., 
       [497,   0,   0, ...,   0,   0,   0],
       [498,   0,   0, ...,   0,   0,   1],
       [499,   0,   0, ...,   0,   0,   1]])

In [229]:
true_last_seq = np.asarray([get_middle(data, n) for n in xrange(*indices[-1])])
# true_last_seq
np.all(true_last_seq == last_protein.values[1:, :].flatten())

True

# Now concatenate all of the proteins together to make the fixed 5 file